<a href="https://colab.research.google.com/github/unt-iialab/INFO5731_Spring2020/blob/master/Assignments/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [Apple iPhone 11](https://www.amazon.com/Apple-iPhone-11-64GB-Unlocked/dp/B07ZPKF8RG/ref=sr_1_13?dchild=1&keywords=iphone+12&qid=1631721363&sr=8-13) on amazon.

(2) Collect the top 10000 User Reviews of the film [Shang-Chi and the Legend of the Ten Rings](https://www.imdb.com/title/tt9376612/reviews?ref_=tt_sa_3) from IMDB.

(3) Collect all the reviews of the top 100 most popular software from [G2](https://www.g2.com/) or [Capterra](https://www.capterra.com/)

(4) Collect the abstracts of the top 10000 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(5) Collect all the information of the 904 narrators in the [Densho Digital Repository](https://ddr.densho.org/narrators/).

(6) Collect the top 10000 tweets by using hashtag ["#blacklivesmatter"](https://twitter.com/hashtag/blacklivesmatter) from Twitter. 


In [ ]:
!pip install tweepy

In [6]:
# Write your code here

import pandas as pd
import tweepy as tw
my_api_key = "LDwXQGW72MGP7gck69WSsIqGK"
my_api_secret = "3vy5DForfcw8lTGQWzUkj42Jfmy0E0xL6FhnVwhqmBOzFHfmgi"

auth = tw.OAuthHandler(my_api_key, my_api_secret)
api = tw.API(auth, wait_on_rate_limit=True)

search_query = "#blacklivesmatter"

def getTweets():
  tweets = tw.Cursor(api.search,
              q=search_query,
              lang="en",
              since="2020-01-10").items(10000)
  reviews = []
  for tweet in tweets:
    reviews.append(tweet)
  return reviews

def createTweetDataFrame():
  tweetReviews = getTweets()
  df = pd.DataFrame()\

  for review in tweetReviews:
    hashtags = []
    try:
        for hashtag in review.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=review.id, tweet_mode='extended').full_text
    except:
        pass

    df = df.append(pd.DataFrame({'username': review.user.name, 
                                               'userlocation': review.user.location,\
                                               'userdescription': review.user.description,
                                               'userverified': review.user.verified,
                                               'date': review.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': review.source}))
    df = df.reset_index(drop=True)
  return df

df = createTweetDataFrame()
df.to_csv('Tweetdata01.csv')
df



,username,userlocation,userdescription,userverified,date,text,hashtags,source
0,Que G🏙,"Harlem, New York",MMTL 〽️〽️💰,False,2022-02-26 18:48:11,"RT @AfricanArchives: 10 years ago today, at on...",None,Twitter for iPhone
1,NIA,"Tyler, TX",sagitterrorist,False,2022-02-26 18:48:09,"RT @AfricanArchives: 10 years ago today, at on...",None,Twitter for iPhone
2,Stephen McCutchan,,Healthy Clergy Make Healthy Congregations. My ...,False,2022-02-26 18:48:03,Learn the concrete steps on how spiritual lead...,None,Buffer
3,Your Aries Comrade (she/her),Georgia Peach 🍑,20 • Aries Sun • Pisces Moon • Libra Rising • ...,False,2022-02-26 18:48:00,RT @angelicaross: Found the @watchallblk app o...,"[BeyondEdBuck, BlackLivesMatter]",Twitter for iPhone
4,Sixes is Watching☥,Mt. Ararat,#66 Morgan State | Arizona Cardinals 6...,False,2022-02-26 18:47:58,"RT @AfricanArchives: 10 years ago today, at on...",None,Twitter for iPhone
...,...,...,...,...,...,...,...,...
9995,justice,,#BlackLivesMatter This is a movement not a mom...,False,2022-02-25 05:00:41,RT @DeccanHerald: A jury found three former Mi...,[GeorgeFloyd],PurpleBot123
9996,alimaG,"Pretoria, South Africa",,False,2022-02-25 05:00:41,RT @Alima1000: The trustees of GEMS have aucti...,"[KKK, MedicalAidRacialProfiling]",Twitter for iPhone
9997,Duh Health,donnamellington@gmail.com,Founded by Sue Saltmarsh Demand Universal Heal...,False,2022-02-25 05:00:40,"RT @marrs_edward: I know there's a war but,\n#...","[BlackLivesMatter, FreeLeonardPeltier, StopAsi...",Twitter for iPad
9998,justice,,#BlackLivesMatter This is a movement not a mom...,False,2022-02-25 05:00:39,RT @OldMainBell: Ding ding* ding Ding &amp; Di...,"[PennState, BlackLivesMatter, JusticeForAll]",PurpleBot123


In [ ]:
!pip install datascience
!pip install numpy
!pip install matplotlib
!pip install pandas
!pip install tensorflow

In [30]:
import pandas as pd
import numpy as np
from google.colab import files
uploaded = files.upload()
import io
RowTweetdata = pd.read_csv(io.BytesIO(uploaded["Tweetdata01.csv"]))
Tweetdata = RowTweetdata
Tweetdata.head(7)


Saving Tweetdata01.csv to Tweetdata01 (1).csv


,Unnamed: 0,username,userlocation,userdescription,userverified,date,text,hashtags,source
0,0,Que G🏙,"Harlem, New York",MMTL 〽️〽️💰,False,2022-02-26 18:48:11,"RT @AfricanArchives: 10 years ago today, at on...",NaN,Twitter for iPhone
1,1,NIA,"Tyler, TX",sagitterrorist,False,2022-02-26 18:48:09,"RT @AfricanArchives: 10 years ago today, at on...",NaN,Twitter for iPhone
2,2,Stephen McCutchan,NaN,Healthy Clergy Make Healthy Congregations. My ...,False,2022-02-26 18:48:03,Learn the concrete steps on how spiritual lead...,NaN,Buffer
3,3,Your Aries Comrade (she/her),Georgia Peach 🍑,20 • Aries Sun • Pisces Moon • Libra Rising • ...,False,2022-02-26 18:48:00,RT @angelicaross: Found the @watchallblk app o...,"['BeyondEdBuck', 'BlackLivesMatter']",Twitter for iPhone
4,4,Sixes is Watching☥,Mt. Ararat,#66 Morgan State | Arizona Cardinals 6...,False,2022-02-26 18:47:58,"RT @AfricanArchives: 10 years ago today, at on...",NaN,Twitter for iPhone
5,5,alimaG,"Pretoria, South Africa",NaN,False,2022-02-26 18:47:49,"""South Africa""\n\nGEMS has the audacity to rep...",NaN,Twitter for iPhone
6,6,Lori Kershaw,NaN,"Animal and alt rock music lover, do unto others",False,2022-02-26 18:47:49,RT @RGarcia_Sports: Trayvon Martin was murdere...,"['RIP', 'BLM', 'BlackLivesMatter']",Twitter Web App


# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [33]:
#Remove noise
import re
Tweetdata = Tweetdata[['date','text']]
Tweetdata["WithoutNoise"] = Tweetdata["text"].str.replace("[^\w\s]", "")
Tweetdata["WithoutNoise"] = Tweetdata["text"].str.replace("RT", "")
Tweetdata["WithoutNoise"] = Tweetdata["WithoutNoise"].apply(lambda item: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', char) for char in item ))
Tweetdata.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


,date,text,WithoutNoise
0,2022-02-26 18:48:11,"RT @AfricanArchives: 10 years ago today, at on...",AfricanArchives 10 years ago today at only...
1,2022-02-26 18:48:09,"RT @AfricanArchives: 10 years ago today, at on...",AfricanArchives 10 years ago today at only...
2,2022-02-26 18:48:03,Learn the concrete steps on how spiritual lead...,Learn the concrete steps on how spiritual lead...
3,2022-02-26 18:48:00,RT @angelicaross: Found the @watchallblk app o...,angelicaross Found the watchallblk app on ...
4,2022-02-26 18:47:58,"RT @AfricanArchives: 10 years ago today, at on...",AfricanArchives 10 years ago today at only...


In [34]:
#Remove numbers
Tweetdata["WithoutNumbers"] = Tweetdata["WithoutNoise"].apply(lambda item: "".join([i for i in item if not i.isdigit()]))
Tweetdata.head()

,date,text,WithoutNoise,WithoutNumbers
0,2022-02-26 18:48:11,"RT @AfricanArchives: 10 years ago today, at on...",AfricanArchives 10 years ago today at only...,AfricanArchives years ago today at only ...
1,2022-02-26 18:48:09,"RT @AfricanArchives: 10 years ago today, at on...",AfricanArchives 10 years ago today at only...,AfricanArchives years ago today at only ...
2,2022-02-26 18:48:03,Learn the concrete steps on how spiritual lead...,Learn the concrete steps on how spiritual lead...,Learn the concrete steps on how spiritual lead...
3,2022-02-26 18:48:00,RT @angelicaross: Found the @watchallblk app o...,angelicaross Found the watchallblk app on ...,angelicaross Found the watchallblk app on ...
4,2022-02-26 18:47:58,"RT @AfricanArchives: 10 years ago today, at on...",AfricanArchives 10 years ago today at only...,AfricanArchives years ago today at only ...


In [36]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
#Remove stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')
Tweetdata['WithoutStopWords'] = Tweetdata['WithoutNumbers'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
Tweetdata.head()

,date,text,WithoutNoise,WithoutNumbers,WithoutStopWords
0,2022-02-26 18:48:11,"RT @AfricanArchives: 10 years ago today, at on...",AfricanArchives 10 years ago today at only...,AfricanArchives years ago today at only ...,AfricanArchives years ago today unarmed Trayvo...
1,2022-02-26 18:48:09,"RT @AfricanArchives: 10 years ago today, at on...",AfricanArchives 10 years ago today at only...,AfricanArchives years ago today at only ...,AfricanArchives years ago today unarmed Trayvo...
2,2022-02-26 18:48:03,Learn the concrete steps on how spiritual lead...,Learn the concrete steps on how spiritual lead...,Learn the concrete steps on how spiritual lead...,Learn concrete steps spiritual leaders integra...
3,2022-02-26 18:48:00,RT @angelicaross: Found the @watchallblk app o...,angelicaross Found the watchallblk app on ...,angelicaross Found the watchallblk app on ...,angelicaross Found watchallblk app Samsung TV ...
4,2022-02-26 18:47:58,"RT @AfricanArchives: 10 years ago today, at on...",AfricanArchives 10 years ago today at only...,AfricanArchives years ago today at only ...,AfricanArchives years ago today unarmed Trayvo...


In [41]:
#Lowercase all texts
Tweetdata['WithoutLowerCase'] = Tweetdata['WithoutStopWords'].apply(lambda x: " ".join(x.lower() for x in x.split()))
Tweetdata.head()


,date,text,WithoutNoise,WithoutNumbers,WithoutStopWords,WithoutLowerCase,Stemming
0,2022-02-26 18:48:11,"RT @AfricanArchives: 10 years ago today, at on...",AfricanArchives 10 years ago today at only...,AfricanArchives years ago today at only ...,AfricanArchives years ago today unarmed Trayvo...,africanarchives years ago today unarmed trayvo...,africanarch year ago today unarm trayvon marti...
1,2022-02-26 18:48:09,"RT @AfricanArchives: 10 years ago today, at on...",AfricanArchives 10 years ago today at only...,AfricanArchives years ago today at only ...,AfricanArchives years ago today unarmed Trayvo...,africanarchives years ago today unarmed trayvo...,africanarch year ago today unarm trayvon marti...
2,2022-02-26 18:48:03,Learn the concrete steps on how spiritual lead...,Learn the concrete steps on how spiritual lead...,Learn the concrete steps on how spiritual lead...,Learn concrete steps spiritual leaders integra...,learn concrete steps spiritual leaders integra...,learn concret step spiritu leader integr anti ...
3,2022-02-26 18:48:00,RT @angelicaross: Found the @watchallblk app o...,angelicaross Found the watchallblk app on ...,angelicaross Found the watchallblk app on ...,angelicaross Found watchallblk app Samsung TV ...,angelicaross found watchallblk app samsung tv ...,angelicaross found watchallblk app samsung tv ...
4,2022-02-26 18:47:58,"RT @AfricanArchives: 10 years ago today, at on...",AfricanArchives 10 years ago today at only...,AfricanArchives years ago today at only ...,AfricanArchives years ago today unarmed Trayvo...,africanarchives years ago today unarmed trayvo...,africanarch year ago today unarm trayvon marti...


In [42]:
#Stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
Tweetdata["Stemming"] = Tweetdata['WithoutLowerCase'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
Tweetdata.head()

,date,text,WithoutNoise,WithoutNumbers,WithoutStopWords,WithoutLowerCase,Stemming
0,2022-02-26 18:48:11,"RT @AfricanArchives: 10 years ago today, at on...",AfricanArchives 10 years ago today at only...,AfricanArchives years ago today at only ...,AfricanArchives years ago today unarmed Trayvo...,africanarchives years ago today unarmed trayvo...,africanarch year ago today unarm trayvon marti...
1,2022-02-26 18:48:09,"RT @AfricanArchives: 10 years ago today, at on...",AfricanArchives 10 years ago today at only...,AfricanArchives years ago today at only ...,AfricanArchives years ago today unarmed Trayvo...,africanarchives years ago today unarmed trayvo...,africanarch year ago today unarm trayvon marti...
2,2022-02-26 18:48:03,Learn the concrete steps on how spiritual lead...,Learn the concrete steps on how spiritual lead...,Learn the concrete steps on how spiritual lead...,Learn concrete steps spiritual leaders integra...,learn concrete steps spiritual leaders integra...,learn concret step spiritu leader integr anti ...
3,2022-02-26 18:48:00,RT @angelicaross: Found the @watchallblk app o...,angelicaross Found the watchallblk app on ...,angelicaross Found the watchallblk app on ...,angelicaross Found watchallblk app Samsung TV ...,angelicaross found watchallblk app samsung tv ...,angelicaross found watchallblk app samsung tv ...
4,2022-02-26 18:47:58,"RT @AfricanArchives: 10 years ago today, at on...",AfricanArchives 10 years ago today at only...,AfricanArchives years ago today at only ...,AfricanArchives years ago today unarmed Trayvo...,africanarchives years ago today unarmed trayvo...,africanarch year ago today unarm trayvon marti...


In [43]:
!pip install textblob

In [47]:
# Lemmatization

from textblob import Word
import nltk
nltk.download('wordnet')

Tweetdata['Lemmatization'] = Tweetdata['Stemming'].apply(lambda x: " ".join([Word(word).lemmatize() for word in str(x).split()]))
Tweetdata.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,date,text,WithoutNoise,WithoutNumbers,WithoutStopWords,WithoutLowerCase,Stemming,Lemmatization
0,2022-02-26 18:48:11,"RT @AfricanArchives: 10 years ago today, at on...",AfricanArchives 10 years ago today at only...,AfricanArchives years ago today at only ...,AfricanArchives years ago today unarmed Trayvo...,africanarchives years ago today unarmed trayvo...,africanarch year ago today unarm trayvon marti...,africanarch year ago today unarm trayvon marti...
1,2022-02-26 18:48:09,"RT @AfricanArchives: 10 years ago today, at on...",AfricanArchives 10 years ago today at only...,AfricanArchives years ago today at only ...,AfricanArchives years ago today unarmed Trayvo...,africanarchives years ago today unarmed trayvo...,africanarch year ago today unarm trayvon marti...,africanarch year ago today unarm trayvon marti...
2,2022-02-26 18:48:03,Learn the concrete steps on how spiritual lead...,Learn the concrete steps on how spiritual lead...,Learn the concrete steps on how spiritual lead...,Learn concrete steps spiritual leaders integra...,learn concrete steps spiritual leaders integra...,learn concret step spiritu leader integr anti ...,learn concret step spiritu leader integr anti ...
3,2022-02-26 18:48:00,RT @angelicaross: Found the @watchallblk app o...,angelicaross Found the watchallblk app on ...,angelicaross Found the watchallblk app on ...,angelicaross Found watchallblk app Samsung TV ...,angelicaross found watchallblk app samsung tv ...,angelicaross found watchallblk app samsung tv ...,angelicaross found watchallblk app samsung tv ...
4,2022-02-26 18:47:58,"RT @AfricanArchives: 10 years ago today, at on...",AfricanArchives 10 years ago today at only...,AfricanArchives years ago today at only ...,AfricanArchives years ago today unarmed Trayvo...,africanarchives years ago today unarmed trayvo...,africanarch year ago today unarm trayvon marti...,africanarch year ago today unarm trayvon marti...


In [48]:
#CSV file
Cleandata01 = Tweetdata[['text','Lemmatization']]
Cleandata01 = Cleandata01.rename(columns={'text': 'RawText', 'Lemmatization': 'CleanText'})
Cleandata01.to_csv('CleanDataAva.csv')
Cleandata01

,RawText,CleanText
0,"RT @AfricanArchives: 10 years ago today, at on...",africanarch year ago today unarm trayvon marti...
1,"RT @AfricanArchives: 10 years ago today, at on...",africanarch year ago today unarm trayvon marti...
2,Learn the concrete steps on how spiritual lead...,learn concret step spiritu leader integr anti ...
3,RT @angelicaross: Found the @watchallblk app o...,angelicaross found watchallblk app samsung tv ...
4,"RT @AfricanArchives: 10 years ago today, at on...",africanarch year ago today unarm trayvon marti...
...,...,...
9995,RT @DeccanHerald: A jury found three former Mi...,deccanherald a juri found three former minneap...
9996,RT @Alima1000: The trustees of GEMS have aucti...,alima the truste gem auction fiduciari duti kk...
9997,"RT @marrs_edward: I know there's a war but,\n#...",marr edward i know war blacklivesmatt plea fre...
9998,RT @OldMainBell: Ding ding* ding Ding &amp; Di...,oldmainbel ding ding ding ding amp ding ding d...


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [53]:
import pandas as pd
import nltk
from nltk import word_tokenize

In [54]:
# Write your code here
#1-Parts of speech
nltk.download('punkt')

nltk.download('averaged_perceptron_tagger')
data = pd.read_csv("CleanDataAva.csv")

PartsOfSpeech = []
for i in data['CleanText']:
    word = word_tokenize(i)
    PartsOfSpeech.append(nltk.pos_tag(word))
PartsOfSpeech

print (PartsOfSpeech)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[[('africanarch', 'NN'), ('year', 'NN'), ('ago', 'RB'), ('today', 'NN'), ('unarm', 'JJ'), ('trayvon', 'NN'), ('martin', 'NN'), ('shot', 'NN'), ('dead', 'JJ'), ('georg', 'NN'), ('zimmerman', 'NN'), ('fuel', 'NN'), ('movement', 'NN'), ('blac', 'NN')], [('africanarch', 'NN'), ('year', 'NN'), ('ago', 'RB'), ('today', 'NN'), ('unarm', 'JJ'), ('trayvon', 'NN'), ('martin', 'NN'), ('shot', 'NN'), ('dead', 'JJ'), ('georg', 'NN'), ('zimmerman', 'NN'), ('fuel', 'NN'), ('movement', 'NN'), ('blac', 'NN')], [('learn', 'NN'), ('concret', 'JJ'), ('step', 'NN'), ('spiritu', 'JJ'), ('leader', 'NN'), ('integr', 'NN'), ('anti', 'VBD'), ('racism', 'NN'), ('ministri', 'NN'), ('spiritu', 'JJ'), ('leadership', 'NN')

In [55]:
PartsOfSpeech

[[('africanarch', 'NN'),
  ('year', 'NN'),
  ('ago', 'RB'),
  ('today', 'NN'),
  ('unarm', 'JJ'),
  ('trayvon', 'NN'),
  ('martin', 'NN'),
  ('shot', 'NN'),
  ('dead', 'JJ'),
  ('georg', 'NN'),
  ('zimmerman', 'NN'),
  ('fuel', 'NN'),
  ('movement', 'NN'),
  ('blac', 'NN')],
 [('africanarch', 'NN'),
  ('year', 'NN'),
  ('ago', 'RB'),
  ('today', 'NN'),
  ('unarm', 'JJ'),
  ('trayvon', 'NN'),
  ('martin', 'NN'),
  ('shot', 'NN'),
  ('dead', 'JJ'),
  ('georg', 'NN'),
  ('zimmerman', 'NN'),
  ('fuel', 'NN'),
  ('movement', 'NN'),
  ('blac', 'NN')],
 [('learn', 'NN'),
  ('concret', 'JJ'),
  ('step', 'NN'),
  ('spiritu', 'JJ'),
  ('leader', 'NN'),
  ('integr', 'NN'),
  ('anti', 'VBD'),
  ('racism', 'NN'),
  ('ministri', 'NN'),
  ('spiritu', 'JJ'),
  ('leadership', 'NN'),
  ('commun', 'NN'),
  ('click', 'NN'),
  ('info', 'NN'),
  ('http', 'NN'),
  ('co', 'NN'),
  ('jofotswpu', 'NN'),
  ('dismantlewhitesupremaci', 'NN'),
  ('thisisamerica', 'NN'),
  ('racialjustic', 'JJ'),
  ('racisminamerica

In [56]:
nouns=0
verbs=0
adjectives=0
adverbs=0

for i in PartsOfSpeech:
    for j in i:
        if 'NN' in j:
            nouns=nouns+1
        if 'VB' in j:
            verbs+=1
        if 'JJ' in j:
            adjectives+=1
        if 'RB' in j:
            adverbs+=1
        
print('#NOUNS are:', nouns)
print('#VERBS are:', verbs)
print('#ADJECTIVES are:', adjectives)
print('#ADVERBS are:', adverbs)

#NOUNS are: 88997
#VERBS are: 3577
#ADJECTIVES are: 26189
#ADVERBS are: 4688


In [ ]:
!pip install benepar

In [58]:
#2-Constituency Parsing
import benepar

benepar.download('benepar_en3')
parser = benepar.Parser("benepar_en3")
for i in data['CleanText']:
    try:
        tree = parser.parse(i)
        print(tree)
    except:
        print("Exception Raised")
        continue

[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


/usr/local/lib/python3.7/dist-packages/torch/distributions/distribution.py:46: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  'with `validate_args=False` to turn off validation.')


Streaming output truncated to the last 5000 lines.
      (VBD broke)
      (NP (NNS busi))
      (VB steal)
      (IN around)
      (RB cigarett)
      (S
        (NP (NP (PRP they)) (JJ broad) (NN daylight))
        (VP (VBP hide) (FW b))))))
(TOP
  (S
    (NP
      (NP (PRP i))
      (VBP releas)
      (NP (NP (JJ new) (NN music)) (NP (NN imafrican))))
    (VP
      (VBD made)
      (S
        (NP (JJ africa) (NN world))
        (VP
          (VB check)
          (RB fam)
          (NP
            (FW http)
            (FW co)
            (FW acczdpttfp)
            (FW blacklivesmatt)
            (FW blackhistorymonth)
            (FW http)
            (FW co)
            (FW jizjrvbuf)))))))
(TOP
  (S
    (NP (NN putin))
    (ADVP (RBR better))
    (VB care)
    (NP
      (VBN forgotten)
      (NP (JJ black) (NN woman))
      (NN power)
      (NN vice)
      (NN presid))
    (ADVP (RB better))
    (VP (VB get) (NP (JJ wrong) (NN side)))
    (FW kamala)
    (FW harri)
    (NN whip)


In [59]:
#2-Dependency Parsing
import pandas as pd
import spacy
data = pd.read_csv("CleanDataAva.csv")

nlp=spacy.load('en_core_web_sm')

for i in data['CleanText']:
    for token in nlp(i):
        print (token.text, '=>', token.dep_, "=>", token.head.text)

Streaming output truncated to the last 5000 lines.
save => ccomp => say
her => dobj => save
from => prep => save
duo => compound => http
http => compound => huwddzfi
co => compound => huwddzfi
huwddzfi => pobj => from
via => prep => save
youtub => compound => blacklivesmatt
blacklivesmatt => pobj => via
foxnew => ROOT => foxnew
politicsl => advmod => foxnew
aurtemu => compound => blacknessinxtra
blacknessinxtra => nsubj => blacklivesmatt
blacklivesmatt => ROOT => blacklivesmatt
blaxktwitt => compound => biden
biden => compound => care
care => compound => thedemocrat
blackpeopl => compound => thedemocrat
thedemocrat => compound => pelo
segregationist => compound => pelo
pelo => dobj => blacklivesmatt
mrandyngo => compound => thiev
san => compound => francisco
francisco => compound => thiev
thiev => nsubj => broke
broke => ROOT => broke
busi => dobj => broke
steal => ccomp => broke
around => prep => steal
cigarett => pobj => around
they => nsubj => broad
broad => advmod => hide
daylight 

In [60]:
#3- Named Entity Recognition

import spacy
from spacy import displacy

NER = spacy.load("en_core_web_sm")

names = 0
organizations = 0
locations = 0
date = 0
nonGPElocations = 0

for i in data["CleanText"]:
    text1= NER(i)
    for word in text1.ents:
        print(word.text,word.label_)
        if word.label_ == "DATE":
            date=date+1
        elif word.label_ == "GPE":
            locations=locations+1
        elif word.label_ == "LOC":
            nonGPElocations=nonGPElocations+1
        elif word.label_ == "ORG":
            organizations=organizations+1
        elif word.label_ == "PERSON":
            names=names+1
        else: 0
    
print("#Names present are :",names) 
print("#Organizations present are:",organizations) 
print("#Locations present are:",locations) 
print("#Dates present are:",date) 
print("#Non GPE Locations present are:",nonGPElocations)

Streaming output truncated to the last 5000 lines.
today DATE
awar chang PERSON
etf ORG
russian NORP
equiti GPE
defenc ORG
holocaustmemorialday DATE
mlk grenfelltow PERSON
salari ORG
adida GPE
year DATE
cumeberechukwu nigeriainvas PERSON
european NORP
russia GPE
mlk grenfelltow PERSON
blacklivesmatt bidenharri oprahmeghanharri PERSON
sacr world rememb post ORG
cumeberechukwu nigeriainvas PERSON
south african NORP
south africa GPE
america GPE
someth sound ORG
alreadi ORG
mlk grenfelltow PERSON
globalsfsu ORG
cumeberechukwu nigeriainvas PERSON
kennethmccun npr PERSON
blacklivesmatt ORG
hatr racism ORG
truewordsrspok ragipsoylu PERSON
foxnew tuckercarlson PERSON
putin PERSON
np ORG
slur NORP
blacklivesmatt np PERSON
bryantterri cookbook PERSON
mani wonder delici recip PERSON
russian NORP
keasi black prosper PERSON
w code bhm http PRODUCT
one CARDINAL
last week DATE
russia GPE
cudjin PERSON
jiminhof vetaffairsdem deptvetaffair ehrmva PERSON
amvetshq moph PERSON
bryantterri cookbook PERSON


**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

**The constituency parsing tree**

The constituency parse tree is based on the formalism of context-free grammars. In this type of tree, the sentence is divided into constituents, that is, sub-phrases that belong to a specific category in the grammar.

**The dependency parsing tree**

A dependency parse tree is the directed graph which has the below features: Root has no Incoming arcs (can only be Head in Head-Dependent pair) Vertices(except Root) should have only one incoming arc (Only one Parent/Head) A Unique path should exist between Root & each vertex in the tree.

Dependency parsing displays only relationships between words and their constitutes while constituency parsing displays the entire sentence structure and relationships. Often dependency parsing is praised for being concise yet informative, but constituency parsing is often easier to read and understand.
The constituency parsing tree is similar to hierarchical relationships while the dependency parsing tree is similar to network relationship.




I used this text as an example to explain these two methods.
Text: An unarmed Trayvon Martin was shot dead by George Zimmerman.
The results of the constituency parsing tree is as follows 
    (NP (DT An) (JJ unarmed) (NNP Trayvon) (NNP Martin))
    (VP
      (VBD was)
      (VP
        (VBN shot)
        (S (ADJP (JJ dead)))
        (PP (IN by) (NP (NNP George) (NNP Zimmerman)))))))
 
In this type of tree, the sentence is divided into constituents, that is, sub-phrases that belong to a specific category in the grammar. For example, Trayvon and Martin are name and they are labeled by “NNP”.
The below shows the result of dependency parsing tree. For example, unarmed is dependent and Martin is head.

An det 0 0 ['Martin', 'shot']
unarmed amod 0 0 ['Martin', 'shot']
Trayvon compound 0 0 ['Martin', 'shot']
Martin nsubjpass 3 0 ['shot']

 
